# **Clone Git Repository**

---

Clone git agar dapat load data langsung dari git repository. Dataset yang digunakan didapat kan dari kaggle: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?select=Books.csv

In [ ]:
!git clone https://github.com/ziszz/book-recommendation.git

# **Import library yang diperlukan**

---



In [ ]:
import pandas as pd
import numpy as np
import zipfile
import glob
import os
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

from zipfile import ZipFile
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# **Data Loading**

---



In [ ]:
books = pd.read_csv('/content/book-recommendation/datasets/Books.csv', encoding='utf-8')
ratings = pd.read_csv('/content/book-recommendation/datasets/Ratings.csv', encoding='utf-8')

In [ ]:
books

In [ ]:
ratings

# **Exploratory Data**

---

## Variabel-variabel pada dataset adalah sebagai berikut:

### **Books.csv**
  * `ISBN`: Kode pengidentifikasian buku yang bersifat unik.
  * `Book-Title`: Judul Buku.
  * `Book-Author`: Nama pengarang buku.
  * `Publisher`: Pihak penerbit buku.
  * `Image-URL-S`: URL yang menautkan ke gambar sampul berukuran small.
  * `Image-URL-M`: URL yang menautkan ke gambar sampul berukuran medium.
  * `Image-URL-L`: URL yang menautkan ke gambar sampul berukuran large.

### **Ratings.csv**
  * `User-ID`: Nomer unik user yang memberikan rating.
  * `ISBN`: Kode pengidentifikasian buku yang bersifat unik.
  * `Book-Rating`: Skor dari rating yang diberikan.


## **Menghitung jumlah Buku, User dan Rating**

In [ ]:
print('Jumlah data buku: ', len(books['ISBN'].unique()))
print('Jumlah data user yang memberikan rating: ', len(ratings['User-ID'].unique()))
print('Jumlah data rating pada buku: ', len(ratings['ISBN'].unique()))

## **Mendapatkan info pada data**

In [ ]:
books.info()

Terlihat dari data buku di atas. Semua kolom data memiliki type data object

In [ ]:
ratings.info()

Sedangkan, untuk data rating terdapat 2 tipe pada data yaitu numerik (int64) dan object.

In [ ]:
book_list = books['Book-Title'].value_counts().keys()
jumlah = books['Book-Title'].value_counts()

book_count = pd.DataFrame({'Book-Title': book_list, 'Jumlah': jumlah}).reset_index(drop=True)
book_count

Pada data terdapat 242135 data unik judul buku. Terlihat bahwa ada beberapa baris dalam data buku memiliki judul buku yang sama. Oleh karena itu, data duplikat perlu dihapus dan hanya memilih baris dengan Judul Buku yang unik.

In [ ]:
rating_list = ratings['Book-Rating'].value_counts().keys()
jumlah = ratings['Book-Rating'].value_counts()

rating_count = pd.DataFrame({'Ratings': rating_list, 'Jumlah': jumlah}).reset_index(drop=True)
rating_count

Dari output di atas, diketahui bahwa nilai maksimum rating adalah 10 dan nilai minimumnya adalah 0. Artinya, skala rating berkisar antara 0 hingga 10.

## **Memeriksa missing value**


In [ ]:
books.isnull().sum()

In [ ]:
ratings.isnull().sum()


Jika dilihat dari data buku dan data rating di atas. Terdapat sedikit missing value pada data buku, sedangkan pada data rating tidak memiliki missing value.

# **Content-Based Filtering**
---
## **Data Preparation**



### **Menghapus data yang tidak diperlukan**
Sistem rekomendasi ini hanya memerlukan data author dan rating sebagai fitur untuk model. Beberapa kolom data seperti `'Year-Of-Publication', 'Publisher', 'Image-URL-M', 'Image-URL-L'` tidak akan digunakan untuk sistem rekomendasi ini. Jadi data tersebut bisa dihapus.

In [ ]:
unused_columns = ['Year-Of-Publication', 'Publisher', 'Image-URL-M', 'Image-URL-L']
books.drop(unused_columns, axis=1, inplace=True)
books

### **Menggabungkan data buku dan rating**

In [ ]:
new_ratings = ratings.merge(books,on='ISBN')
new_ratings = new_ratings.groupby('Book-Title').sum()['Book-Rating'].reset_index()
new_ratings.rename(columns={'Book-Rating':'Num-Ratings'}, inplace=True)

In [ ]:
new_books = pd.DataFrame({'Book-Title': books['Book-Title'].unique()})
new_books = pd.merge(new_books, new_ratings, on='Book-Title', how='left')
new_books = new_books.merge(books,on='Book-Title').drop_duplicates('Book-Title')
new_books

### **Mengatasi missing value**

In [ ]:
new_books.isnull().sum()

In [ ]:
new_books = new_books.dropna()
new_books.shape

In [ ]:
new_books.isnull().sum()

### **Menyeleksi data**
Data yang akan digunakan yaitu data buku dengan total skor rating dari tiap buku di atas 500. 

In [ ]:
final_books = new_books[new_books['Num-Ratings'] > 50]
final_books.drop(['ISBN', 'Num-Ratings'], axis=1, inplace=True)
final_books

## **Model Development**
### **Tfid Vectorizer**

In [ ]:
data = final_books
data.sample(5)

In [ ]:
tfid = TfidfVectorizer(token_pattern=r"(?u)\b\w\w+\b\s+\w+")
tfid.fit(data['Book-Author']) 

tfid.get_feature_names() 

### **Transformasi data kedalam bentuk matriks**

In [ ]:
tfidf_matrix = tfid.fit_transform(data['Book-Author']) 
tfidf_matrix.shape

### **Menghitung Cosine Similarity**

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim

In [ ]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['Book-Title'], columns=data['Book-Title'])

### **Mendapatkan rekomendasi buku**
Mendapatkan rekomendasi buku berdasarkan author yang sama dengan buku yang telah dibaca oleh user.

In [ ]:
def book_recommendations(book_name, similarity_data=cosine_sim_df, items=data, k=5):
  index = similarity_data[book_name].to_numpy().argpartition(range(-1, -(k+1), -1))[::-1]
  closest = similarity_data.columns[index[:k+2]]
  closest = closest.drop(book_name, errors='ignore')

  return pd.DataFrame(closest).merge(items).head(k)

In [ ]:
book_recommendations('She\'s Come Undone (Oprah\'s Book Club (Paperback))')

# **Collaborative Filtering**

---

## **Data Preparation**

### **Menggabungkan data buku dan rating**
Tidak seperti pada teknik Content-Based Filtering. Data yang digunakan di teknik Collaborative Filtering kali ini tidak memerlukan data `Book-Author, Num-Ratings,` dan `ISBN`. Sebab pada teknik ini hanya menggunakan rating sebagai acuan sistem rekomendasi.

In [ ]:
df = ratings
df = df.merge(new_books, on='ISBN')
df.drop(['Num-Ratings', 'Book-Author', 'ISBN'], axis=1, inplace=True)

### **Menyandikan fitur**
Membuat penyandian untuk fitur `User-ID` dan `Book-Title` menjadi dalam bentuk index

In [ ]:
user_ids = df['User-ID'].unique().tolist()
user2encoded = {x: i for i, x in enumerate(user_ids)}
encoded2user = {i: x for i, x in enumerate(user_ids)}

In [ ]:
book_title = df['Book-Title'].unique().tolist()
book2encoded = {x: i for i, x in enumerate(book_title)}
encoded2book = {i: x for i, x in enumerate(book_title)}

In [ ]:
df['User-Encoded'] = df['User-ID'].map(user2encoded)
df['Book-Encoded'] = df['Book-Title'].map(book2encoded)

In [ ]:
num_users = len(user2encoded)
print(num_users)
 
num_books = len(encoded2book)
print(num_books)

df['Book-Rating'] = df['Book-Rating'].values.astype(np.float32)
 
min_rating = min(df['Book-Rating'])
max_rating = max(df['Book-Rating'])

print(f'Number of User: {num_users}, Number of Books: {num_books}, Min Rating: {min_rating}, Max Rating: {max_rating}')

In [ ]:
df

### **Normalisasi data rating**
Melakukan transformasi pada data fitur `Book-Rating`. MinMaxScaler mentransformasikan fitur dengan menskalakan setiap fitur ke rentang tertentu. Library ini menskalakan dan mentransformasikan setiap fitur secara individual sehingga berada dalam rentang yang diberikan pada set pelatihan, pada library ini memiliki range default antara nol dan satu.

In [ ]:
x = df[['User-Encoded', 'Book-Encoded']].values
y = df['Book-Rating'].values
y = y.reshape(-1, 1)

In [ ]:
scaler = MinMaxScaler()
norm_y = scaler.fit_transform(y)
norm_y = norm_y.reshape(1, -1)[0]

### **Split dataset**

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, norm_y, test_size=0.2, random_state=123)

### **Membuat data pipeline**

In [ ]:
def create_dataset(x, y, batch_size, buffer_size=None, shuffle=True):
  ds = tf.data.Dataset.from_tensor_slices((x, y))

  if shuffle:
    ds = ds.shuffle(buffer_size)

  ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)

  return ds

In [ ]:
batch_size = 128
buffer_size = len(x)

train_ds = create_dataset(x_train, y_train, batch_size, buffer_size)
val_ds = create_dataset(x_val, y_val, batch_size, shuffle=False)

## **Model Development**
### **Membuat model**

In [ ]:
class RecommenderNet(tf.keras.Model):
  def __init__(self, num_users, num_books, embedding_size, **kwargs):
    super(RecommenderNet, self).__init__(**kwargs)

    self.num_users = num_users
    self.num_books = num_books
    self.embedding_size = embedding_size
    self.user_embedding = layers.Embedding(
        num_users,
        embedding_size,
        embeddings_initializer='he_normal',
        embeddings_regularizer=keras.regularizers.l2(1e-3),
    )
    self.user_bias = layers.Embedding(num_users, 1)
    self.books_embedding = layers.Embedding(
        num_books,
        embedding_size,
        embeddings_initializer='he_normal',
        embeddings_regularizer=keras.regularizers.l2(1e-3),
    )
    self.books_bias = layers.Embedding(num_books, 1)

  def call(self, inputs):
    user_vector = self.user_embedding(inputs[:, 0])
    user_bias = self.user_bias(inputs[:, 0])
    books_vector = self.books_embedding(inputs[:, 1])
    books_bias = self.books_bias(inputs[:, 1])

    dot_user_books = tf.tensordot(user_vector, books_vector, 2)

    x = dot_user_books + user_bias + books_bias

    return tf.nn.sigmoid(x)

In [ ]:
embedding_size = 64

model = RecommenderNet(num_users, num_books, embedding_size)
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

### **Melatih model**

In [ ]:
history = model.fit(
    train_ds,
    epochs = 10,
    validation_data = val_ds,
    verbose=1,
)